**Problem 1: Simple Sampling**

You are not allowed to use sampling libraries/functions. But you can use rand() call to generate a pseudo-uniform value in [0,1]; you can also use a library that computes the pdf(x|params). make sure to recap first Rejection Sampling and Inverse Transform Sampling

A. Implement simple sampling from continuous distributions: uniform (min, max, sample_size) and gaussian (mu, sigma, sample_size)

B. Implement sampling from a 2-dim Gaussian Distribution (2d mu, 2d sigma, sample_size)

C. Implement wihtout-replacement sampling from a discrete non-uniform distribution (given as input) following the Steven's method described in class ( paper ). Test it on desired sample sizes N significantly smaller than population size M (for example N=20 M=300)

In [1]:
import random

def sample_uniform(a, b, sample_size):
    """
    Generates samples from a Uniform(a, b) distribution.

    Args:
        a (float): Lower bound.
        b (float): Upper bound.
        sample_size (int): Number of samples to generate.

    Returns:
        list of float: Uniform samples.
    """
    samples = []
    for _ in range(sample_size):
        u = random.random()  # rand() returns value in [0,1)
        sample = a + (b - a) * u
        samples.append(sample)
    return samples

# Example usage:
print("Uniform samples:", sample_uniform(0, 10, 5))

Uniform samples: [6.987538055619814, 2.6119309767933196, 9.453894979763586, 9.007527242424382, 2.542066158250824]


In [2]:
import math

def sample_gaussian(mu, sigma, sample_size):
    """
    Generates samples from a Gaussian distribution N(mu, sigma^2)
    using the Box-Muller transform.

    Args:
        mu (float): Mean.
        sigma (float): Standard deviation.
        sample_size (int): Number of samples to generate.

    Returns:
        list of float: Gaussian samples.
    """
    samples = []
    # Process two samples per iteration using Box-Muller:
    for _ in range(sample_size // 2):
        u1 = random.random()
        u2 = random.random()
        # Avoid taking log of zero:
        if u1 == 0:
            u1 = 1e-10
        z0 = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)
        z1 = math.sqrt(-2 * math.log(u1)) * math.sin(2 * math.pi * u2)
        samples.append(mu + sigma * z0)
        samples.append(mu + sigma * z1)
    # If sample_size is odd, generate one additional sample:
    if sample_size % 2 != 0:
        u1 = random.random()
        u2 = random.random()
        if u1 == 0:
            u1 = 1e-10
        z0 = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)
        samples.append(mu + sigma * z0)
    return samples

# Example usage:
print("Gaussian samples:", sample_gaussian(0, 1, 5))

Gaussian samples: [-1.2823551358522043, -0.3916149557147224, 2.0250496247006513, -1.0520974229953688, -0.5949501315390834]


In [3]:
def sample_gaussian_2d(mu_vector, sigma_vector, sample_size):
    """
    Generates samples from a 2D Gaussian distribution with independent components.

    Args:
        mu_vector (list or tuple of floats): [mu_x, mu_y].
        sigma_vector (list or tuple of floats): [sigma_x, sigma_y].
        sample_size (int): Number of 2D samples to generate.

    Returns:
        list of [float, float]: List of 2D sample points.
    """
    samples = []

    # For each sample, generate 2 independent standard normal numbers:
    for _ in range(sample_size):
        # We can use the Box-Muller transform to generate a pair; here we only need 2 numbers.
        u1 = random.random()
        u2 = random.random()
        if u1 == 0:
            u1 = 1e-10
        z1 = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)
        # For the second normal, either call Box-Muller again or use the orthogonal value:
        # For clarity, generate a new pair for each sample
        u3 = random.random()
        u4 = random.random()
        if u3 == 0:
            u3 = 1e-10
        z2 = math.sqrt(-2 * math.log(u3)) * math.cos(2 * math.pi * u4)

        # Transform to the desired mean and standard deviation:
        sample_x = mu_vector[0] + sigma_vector[0] * z1
        sample_y = mu_vector[1] + sigma_vector[1] * z2
        samples.append([sample_x, sample_y])

    return samples

# Example usage:
mu_2d = [0, 0]
sigma_2d = [1, 1]
print("2D Gaussian samples:", sample_gaussian_2d(mu_2d, sigma_2d, 5))

2D Gaussian samples: [[-0.962700359874964, -0.6487905857268136], [-0.8560819786230847, -0.34007792571958795], [0.9278813267525409, 1.4630738821801952], [-0.5644163262626185, -0.8470550936827791], [0.5868602180560467, 1.3332663475644468]]


In [4]:
def weighted_sample_without_replacement(weights, sample_size):
    """
    Samples indices without replacement from a list of weights using
    the method based on exponential variates (often called Steven's method).

    Args:
        weights (list of floats): Non-negative weights for the population (length M).
        sample_size (int): Number of items to sample (N), N << M.

    Returns:
        list of int: Selected indices (without replacement).
    """
    keys = []
    # Compute a key for every element.
    for i, w in enumerate(weights):
        # To avoid division by zero, handle zero-weight items appropriately.
        # Here we simply assign an infinite key so they will not be selected.
        if w <= 0:
            key = float('inf')
        else:
            u = random.random()  # Uniform in [0,1)
            # To avoid log(0) in the extremely unlikely event u==0:
            if u == 0:
                u = 1e-10
            key = -math.log(u) / w
        keys.append((key, i))

    # Sort items by their keys (ascending order) and select the sample_size smallest keys.
    keys.sort(key=lambda x: x[0])
    selected_indices = [index for (_, index) in keys[:sample_size]]
    return selected_indices

# Test scenario: N=20, M=300. We generate a synthetic weight vector.
M = 300
N = 20
# For example, let the weights be random positive numbers (not necessarily summing to 1)
population_weights = [random.random() + 0.1 for _ in range(M)]  # adding 0.1 to avoid zero weights

selected = weighted_sample_without_replacement(population_weights, N)
print("Selected indices (without replacement):", selected)

Selected indices (without replacement): [8, 176, 62, 224, 77, 177, 18, 13, 58, 136, 74, 86, 0, 111, 126, 263, 46, 145, 44, 188]


**Problem 2: Conditional Sampling**

Implement Gibbs Sampling for a multidim gaussian generative joint, by using the conditionals which are also gaussian distributions . The minimum requirement is for joint to have D=2 variables and for Gibbs to alternate between the two.

In [5]:
import random
import math

def sample_gaussian_value(mu, sigma):
    """
    Generate a single sample from a Gaussian distribution N(mu, sigma^2)
    using the Box–Muller transform.
    """
    u1 = random.random()
    u2 = random.random()
    # Guard against zero for u1 (extremely unlikely)
    if u1 == 0:
        u1 = 1e-10
    z = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)
    return mu + sigma * z

def gibbs_sampling_2d(mu, sigma, rho, num_samples, burn_in=100):
    """
    Perform Gibbs sampling for a 2-dimensional Gaussian distribution.

    The joint distribution:
         (X, Y) ~ N( [mu_x, mu_y], Σ )
    where Σ is defined as:
         [sigma_x^2        rho*sigma_x*sigma_y]
         [rho*sigma_x*sigma_y   sigma_y^2     ]

    The conditionals are:
      X|Y=y ~ N(mu_x + rho*(sigma_x/sigma_y)*(y-mu_y), (1-rho^2)*sigma_x^2)
      Y|X=x ~ N(mu_y + rho*(sigma_y/sigma_x)*(x-mu_x), (1-rho^2)*sigma_y^2)

    Args:
        mu: list or tuple of two floats [mu_x, mu_y]
        sigma: list or tuple of two floats [sigma_x, sigma_y]
        rho: float, correlation coefficient between X and Y, must be in (-1, 1)
        num_samples: int, number of samples to collect (after burn-in)
        burn_in: int, number of burn-in iterations (default is 100)

    Returns:
        List of [x, y] samples from the joint distribution.
    """

    # Initialize the chain. We can start at the mean.
    x = mu[0]
    y = mu[1]

    samples = []

    # Total iterations: burn_in + actual samples
    total_iterations = burn_in + num_samples

    for iteration in range(total_iterations):
        # Sample X given current Y:
        mean_x_cond = mu[0] + rho * (sigma[0]/sigma[1]) * (y - mu[1])
        var_x_cond = (1 - rho**2) * sigma[0]**2
        x = sample_gaussian_value(mean_x_cond, math.sqrt(var_x_cond))

        # Sample Y given updated X:
        mean_y_cond = mu[1] + rho * (sigma[1]/sigma[0]) * (x - mu[0])
        var_y_cond = (1 - rho**2) * sigma[1]**2
        y = sample_gaussian_value(mean_y_cond, math.sqrt(var_y_cond))

        # After burn_in, collect the sample.
        if iteration >= burn_in:
            samples.append([x, y])

    return samples

# --- Example Usage ---

# Parameters for the joint Gaussian:
mu = [0.0, 0.0]
sigma = [1.0, 1.0]  # Standard deviations for X and Y
rho = 0.8           # Correlation coefficient

# Number of samples we want to collect (after burn-in)
num_samples = 1000
burn_in = 200

# Run Gibbs Sampling
samples = gibbs_sampling_2d(mu, sigma, rho, num_samples, burn_in)

# Print a few samples to check
for i, sample in enumerate(samples[:10]):
    print(f"Sample {i+1}: X = {sample[0]:.4f}, Y = {sample[1]:.4f}")

Sample 1: X = 0.6713, Y = 0.5571
Sample 2: X = 0.1021, Y = 1.5488
Sample 3: X = 0.8961, Y = 0.3792
Sample 4: X = -0.2160, Y = 0.2510
Sample 5: X = -0.9930, Y = 0.7896
Sample 6: X = 1.0555, Y = 0.2000
Sample 7: X = -0.0766, Y = 0.5402
Sample 8: X = 0.0592, Y = 0.6796
Sample 9: X = 0.4553, Y = 0.7211
Sample 10: X = 0.1736, Y = -0.7669


**Problem 3: Implement your own baby-LDA**

Implement your own LDA using Gibbs Sampling, following this paper and this easy-to-read book . Gibbs Sampling is a lot slower than EM alternatives, so this can take some time; use a smaller sample of docs/words at first.

20NG train dataset 11280 docs x 53000 words
Small sonnet dataset (one per line) 154 docs x 3092 words

In [ ]:
import random

def discrete_sample(probabilities):
    """
    Sample an index from a discrete distribution given by a list
    of nonnegative (unnormalized) probabilities.
    """
    total = sum(probabilities)
    r = random.uniform(0, total)
    cumulative = 0.0
    for i, p in enumerate(probabilities):
        cumulative += p
        if cumulative >= r:
            return i
    return len(probabilities) - 1  # fallback

def baby_lda(DOCS, Vocab, K, alpha, beta, iterations):
    """
    A simple LDA implementation using Gibbs sampling.

    Parameters:
      DOCS: List of documents; each document is a list of word indices (integers) in [0, W-1].
      Vocab: List of strings; the vocabulary of size W.
      K: int; number of topics.
      alpha: float; hyperparameter for the document-topic Dirichlet prior.
      beta: float; hyperparameter for the topic-word Dirichlet prior.
      iterations: int; number of Gibbs sampling iterations.

    Returns:
      Z: List (length N) of lists of topic assignments for each word.
      A: Document-topic count matrix (N x K).
      B: Topic-word count matrix (K x W).
    """
    N = len(DOCS)         # number of documents
    W = len(Vocab)        # vocabulary size

    # Initialize document-topic count matrix A: each document starts with the prior alpha for every topic.
    A = [[alpha for _ in range(K)] for _ in range(N)]

    # Initialize topic-word count matrix B: each topic starts with the beta prior for every word.
    B = [[beta for _ in range(W)] for _ in range(K)]

    # BSUM: for each topic, the total count over the vocabulary.
    BSUM = [beta * W for _ in range(K)]

    # Z: topic assignments for each word in each document.
    # We initialize with -1 to indicate that no topic is assigned initially.
    Z = [[-1 for _ in doc] for doc in DOCS]

    # Gibbs sampling iterations.
    for it in range(iterations):
        for d, doc in enumerate(DOCS):
            for i, w in enumerate(doc):
                current_topic = Z[d][i]
                # If a topic was already assigned, subtract its count.
                if current_topic != -1:
                    A[d][current_topic] -= 1
                    B[current_topic][w] -= 1
                    BSUM[current_topic] -= 1

                # Compute the unnormalized conditional probability for each topic.
                topic_probs = []
                for k in range(K):
                    # The probability is the product of:
                    #   (doc-topic count for topic k) and (topic-word probability for word w)
                    # Note: B[k][w] / BSUM[k] is the probability of word w under topic k.
                    prob = A[d][k] * (B[k][w] / BSUM[k])
                    topic_probs.append(prob)

                # Sample a new topic from the computed discrete distribution.
                new_topic = discrete_sample(topic_probs)

                # Update the topic assignment and counts.
                Z[d][i] = new_topic
                A[d][new_topic] += 1
                B[new_topic][w] += 1
                BSUM[new_topic] += 1

        print(f"Iteration {it+1} completed.")

    return Z, A, B

# ---------------------------
# Example usage on a toy dataset:
if __name__ == '__main__':
    # Example vocabulary (for demonstration purposes)
    Vocab = ["apple", "banana", "cat", "dog"]
    # Example documents represented by lists of word indices (each index points into Vocab)
    DOCS = [
        [0, 1, 0, 2, 3, 0],   # e.g., "apple banana apple cat dog apple"
        [1, 3, 3, 2, 2, 1],   # e.g., "banana dog dog cat cat banana"
        [0, 1, 3, 3, 0, 2]    # e.g., "apple banana dog dog apple cat"
    ]

    # Set number of topics, e.g., 2 (in a larger example you might use K=6 or more)
    K = 2
    # Hyperparameters for the Dirichlet priors:
    alpha = 5.0
    beta = 2.0
    # Number of Gibbs sampling iterations:
    iterations = 100

    # Run the baby-LDA Gibbs sampler.
    Z, A, B = baby_lda(DOCS, Vocab, K, alpha, beta, iterations)

    # Display the resulting topic assignments for each document.
    print("\nFinal topic assignments (each sublist corresponds to a document):")
    for d, topics in enumerate(Z):
        print(f"Document {d+1}: {topics}")

    # Display the document-topic counts.
    print("\nDocument-topic matrix (A):")
    for d, counts in enumerate(A):
        print(f"Document {d+1}: {counts}")

    # Display the topic-word counts with top words for each topic.
    print("\nTopic-word distributions (B):")
    for k, word_counts in enumerate(B):
        # Sort words by count (high to low)
        sorted_words = sorted([(Vocab[w], word_counts[w]) for w in range(len(Vocab))],
                              key=lambda x: x[1],
                              reverse=True)
        print(f"Topic {k+1}: {sorted_words}")

    # Note:
    # In practice one might also generate wordclouds per topic, for instance using the matplotlib wordcloud package,
    # but here we simply print the top words by count.